In [ ]:
from cedne import simulator
from cedne import optimizer
from cedne import utils
import numpy as np

In [ ]:
triads = utils.return_triads()
G = triads['030T']
weights = {(1, 3): -3., (3, 2): -1, (1, 2): -3}

input_nodes = [1]
gains = {node:1.0 for node in G.nodes}
tconstants = [5, 5, 1]
time_constants = {n:t for n,t in zip(G.nodes, tconstants)}
initial_rates = [0., 0., 0.]
max_t = 90
time_points = np.linspace(0, max_t, 451)

inp1_value = 1
input_value = {t:inp1_value*np.sin((t/max_t)*2*np.pi)+np.random.normal(0, 0.1) for t in time_points}
inp_vals = [input_value[t] for t in time_points]
input1= simulator.TimeSeriesInput(input_nodes, input_value)

inputs = [input1]

rate_model = simulator.RateModel(G, input_nodes, weights, gains, time_constants, static_nodes=input_nodes, time_points=time_points, inputs=inputs)
real = rate_model.simulate()

f = utils.plot_simulation_results((rate_model, inputs, real), twinx=False)

In [ ]:
real_sim = rate_model.copy()

In [ ]:
node_parameter_bounds =  {'gain': {rn:(1, 5) for n,rn in rate_model.nodes.items() if not n in input_nodes}, 'time_constant': {rn:(1, 5) for n,rn in rate_model.nodes.items() if not n in input_nodes}}
edge_parameter_bounds = {'weight': {e:(-5, 5) for e in rate_model.edges}}
vars_to_fit = [rn for (n, rn) in rate_model.nodes.items() if not n in input_nodes]
## Setting parameter bounds for the paramters of interest and set the rest to default to simulate. Use a noisy output to fit.
o = optimizer.OptunaOptimizer(rate_model, real, optimizer.mean_squared_error, node_parameter_bounds, edge_parameter_bounds, vars_to_fit, num_trials=1e3)
best_params, best_model = o.optimize()
best_fit = best_model.simulate()
f = utils.plot_simulation_results((best_model, inputs, best_fit), twinx=False)

In [ ]:
f = utils.compare_simulation_results((rate_model, inputs, real), (best_model, inputs, best_fit), twinx=False)

In [ ]:
real_sim.node_parameters, real_sim.edge_parameters

In [ ]:
best_model.node_parameters, best_model.edge_parameters